In [1]:
!pip install datasets evaluate transformers seqeval pandas accelerate

In [2]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import evaluate
import datasets
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification
import torch

COLAB = 'google.colab' in sys.modules
if COLAB:
    torch.cuda.is_available
COLAB

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512,garbage_collection_threshold:0.9"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
if not COLAB:
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:80% !important; }</style>"))
    %load_ext autoreload
    %autoreload 2

    sys.path.append("../../")
else:
    if not os.path.exists("ner"):
        from google.colab import drive
        drive.mount("/content/drive")

        !cp -r /content/drive/MyDrive/colab_folder/notebooks .

         # with current datasets package (2.4) create_train_test_split_hf_dataset does not work

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
if COLAB:
    %load_ext tensorboard

In [6]:

tru_label = ['O',
              'B-cond',
              'I-cond',
              'B-des',
              'I-des',
              'B-subj',
              'I-subj',
              "B-group_A",
              "I-group_A",
              "B-group_B",
              "I-group_B",
              "B-group_C",
              "I-group_C",
              'B-group_D',
              'I-group_D']

id2label = {i:label for i, label in enumerate(tru_label) }
label2id = {label:i for i, label in enumerate(tru_label) }

#### Reading the datasest

In [7]:

DATA_PATH = Path("/content/drive/MyDrive/colab_folder/datasets")
MODELS_BASE_PATH = Path("/content/drive/MyDrive/colab_folder/models")


In [8]:
import json
dataset_date = "2023-11-20"

def read_file(DATA_PATH, doc):
    file_path = doc + '.json'
    file_name = os.path.join(DATA_PATH, file_path)

    with open(file_name, "r", encoding="utf-8") as f:
        return json.load(f)

In [12]:
file_names = ["final_labelled_abstract_22_11_2023_2.0","final_labelled_abstract_method_22_11_2023_2.0", "final_labelled_abstract_method_50_22_11_2023_2.0", "final_labelled_abstract_method_500_22_11_2023_2.0" ]
abstract= read_file(DATA_PATH, file_names[0])
abstract_methods = read_file(DATA_PATH, file_names[1])
splitted_500_abstract_methods_context = read_file(DATA_PATH, file_names[3])
splitted_50_records_method_context =read_file(DATA_PATH, file_names[2])




In [10]:
json.dumps(abstract_methods[0])

'{"pmid": "16960863", "ner_tags": [0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 7, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 9, 0, 0, 7, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 9, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
json.dumps(splitted_50_records_method_context[12])


'{"pmid": "16960863", "ner_tags": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "tokens": ["had", "at", "least", "one", "safety", "measurement", "after", "baseline", "were", "included", "in", "safety", "analyses", ".", "mainefficacy", "analyses", "were", "based", "on", "an", "intention-to-treat", "and", "retrieved", "dropout", "(", "itt", "+", "rdo", ")", "population", ",", "defined", "as", "all", "randomized", "patients", "who", "received", "at", "least", "one", "dose", "of", "study", "medication", "and", "had", "a", "least", "one", "assessment", "on", "study", "drug"]}'

#### FUNCTIONS

In [14]:
label_dict = {'O': 0, 'B-condition': 1, 'I-condition': 2,
              'B-design': 3, 'I-design': 4,
              'B-subjects': 5,'I-subjects': 6,
              "B-group_A": 7, "I-group_A": 8,
              "B-group_B": 9, "I-group_B": 10,
              "B-group_C": 11,"I-group_C": 12,
              'B-group_D': 13,'I-group_D': 14
              }
def tokenize_align_labels(example, label_all_tokens = True):
    # tokenized_input = tokenizer(example['tokens'], truncation = True, is_split_into_words=True, max_length = 512)
    tokenized_input = tokenizer(example['tokens'], truncation = True, is_split_into_words=True)

    labels = []
    for i , label in enumerate(example['ner_tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None

        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_ids!= previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input['labels'] = labels
    return tokenized_input

from collections import Counter

def get_key(val):
    for key, value in label_dict.items():
        if val == value:
            return key
    return "key doesn't exist"




def count_values(lst):
    # Use Counter to count occurrences of each value in the list
    value_counts = Counter(lst)

    # Convert the Counter to a dictionary for a more readable output
    count_dict = dict(value_counts)

    return count_dict

def explore_annotations(dataset):
    all_annotations = []

    for records in dataset:
        temp_tag = records['ner_tags']
        all_annotations.extend(temp_tag)
    print(f'There are {len(all_annotations)} annotations in the dataset')

    return [get_key(x) for x in all_annotations]

def normalize_tag_name(tag):
    if tag in [ 'condition', 'condition_1', 'condition_2', 'condition_3', 'condition_4', 'condition_5', 'condition_6', 'condition_7']:
        tag= 'condition'
    elif tag in ['design', 'design_1', 'design_2', 'design_3', 'design_4', 'design_5']:
        tag = 'design'
    elif tag in ['subject_1', 'subject_2', 'subjects', 'subjects_1', 'subjects_2', 'subjects_3', 'subjects_4', 'subjects_5', 'subjekts', 'sujects']:
        tag = 'subjects'
    elif tag in [ 'group_A', 'group_A_1', 'group_A_2', 'group_A_3', 'group_A_4']:
        tag = 'group_A'
    elif tag in ['groub_B_1', 'group_B', 'group_B _2', 'group_B_1', 'group_B_2', 'group_B_3', 'group_B_4' ]:
        tag = 'group_B'
    elif tag in [ 'group_C', 'group_C_1', 'group_C_2', 'group_C_3' ]:
        tag = 'group_C'
    elif tag in [ 'group_D', 'group_D_1', 'group_D_2', 'group_D_3']:
        tag = 'group_D'
    return tag
def create_datasets(doc_dict, labels):
    temp = Dataset.from_pandas(pd.DataFrame(doc_dict))
    temp = temp.cast_column("ner_tags", datasets.Sequence(datasets.ClassLabel(names=labels)))
    return temp

def train_test_split(data_set, train_test_size, validation_size):
    from datasets import DatasetDict
    train_testvalid = data_set.train_test_split(test_size=train_test_size)

    # Split the test to half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=validation_size)

    #Gather everything into one dataset dictionary
    train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

    return train_test_valid_dataset



#### hyperparameters

In [15]:
model_checkpoint = "michiyasunaga/BioLinkBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length = 512)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, ignore_mismatched_sizes=True, id2label = id2label, label2id = label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = evaluate.load('seqeval')

In [17]:
from typing import Tuple
from sklearn.metrics import precision_score, recall_score
from functools import partial

def token_f1_cond(true, pred, labels):
    class_scores = zip(labels, precision_score(true, pred, labels=labels, average=None, zero_division=True),
                       recall_score(true, pred, labels=labels, average=None))
    result = {label: {"f1": get_f1(prec, rec), "p": prec, "r": rec} for label, prec, rec in class_scores}
    return result
def get_f1(prec, rec):
    return 2 * prec * rec / (prec + rec)

def compute_metrics_by_token_swt(p: Tuple[list, list], label_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [label_list[p] for prediction, label in zip(predictions, labels) for (p, l) in
                        zip(prediction, label) if l != -100]
    true_labels = [label_list[l] for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label)
                   if l != -100]

    metrics_report = token_f1_cond(true_labels, true_predictions, label_list)
    metrics_report_f1 = {"f1_" + k: v["f1"] for k, v in metrics_report.items() if k != "O"}
    return metrics_report_f1
compute_metrics_by_token_swt = partial(compute_metrics_by_token_swt, label_list=tru_label)


In [18]:
from transformers import TrainingArguments, Trainer

model_max_length = 512

if COLAB:
    batch_size = 16 # for p-100 16 is ok. For T4: 12
    if model_max_length <= 384:
      # at least 15 GB gpu
        batch_size = 16
    gradient_accumulation_steps = 1
    num_train_epochs = 8
else:
    batch_size = 4
    if model_max_length==256:
        batch_size = 9

    gradient_accumulation_steps = 4
    num_train_epochs = 5

learning_rate = 5e-5
weight_decay = 0.003

print(f"bs: {batch_size}, model_max_length: {model_max_length}, gradient_acc_steps: {gradient_accumulation_steps}, \
n_epochs: {num_train_epochs}, lr: {learning_rate}")

bs: 16, model_max_length: 512, gradient_acc_steps: 1, n_epochs: 8, lr: 5e-05


### ABSTRACT ONLY

In [19]:
for token, label in zip(abstract[0]['tokens'], abstract[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

In [20]:
for token, label in zip(abstract[10]['tokens'], abstract[10]['ner_tags']):
    print(f"{token:_<40} {label}")

five-year_______________________________ 0
follow-up_______________________________ 0
of______________________________________ 0
early___________________________________ 0
lisuride________________________________ 0
and_____________________________________ 0
levodopa________________________________ 0
combination_____________________________ 0
therapy_________________________________ 0
versus__________________________________ 0
levodopa________________________________ 0
monotherapy_____________________________ 0
in______________________________________ 0
de______________________________________ 0
novo____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
the_____________________________________ 0
french__________________________________ 0
lisuride________________________________ 0
study___________________________________ 0
group___________________________________ 0
.__________

##### count the annotations

In [21]:
tags = explore_annotations(abstract)
import pandas as pd
result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 565899 annotations in the dataset


,label_id,count
0,O,533739
1,B-group_A,4820
10,I-condition,4648
7,B-group_B,3778
5,I-design,3711
2,B-subjects,3307
6,I-group_A,2739
9,B-condition,2044
11,I-group_B,2029
3,I-subjects,1839


In [22]:
len(abstract)

1549

#### Creating Huggingface Dataset

In [23]:

abstract_hf_ds = create_datasets(abstract, tru_label)

Casting the dataset:   0%|          | 0/1549 [00:00<?, ? examples/s]

In [24]:
abstract_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [25]:
abstract_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [26]:
abstract_hf_ds_train_test_valid = train_test_split(abstract_hf_ds, train_test_size=0.2, validation_size=0.5)


In [27]:
abstract_hf_ds_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
})

In [28]:
tags = abstract_hf_ds_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### Tokenizer Example and Testing

In [29]:
example_text = abstract_hf_ds_train_test_valid['train'][0]
example_text['tokens']

['to',
 'compare',
 'the',
 'efficacy',
 'of',
 'metformin',
 'with',
 'insulin',
 'in',
 'diabetes',
 'mellitus',
 'in',
 'terms',
 'of',
 'fetomaternal',
 'outcome',
 '-',
 'discussion',
 'needed',
 'upon',
 'quality',
 'of',
 'the',
 'reporting',
 '.']

In [30]:

tokenized_input = tokenizer(example_text['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

words_ids = tokenized_input.word_ids()
words_ids

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 14,
 14,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 None]

In [31]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
tokens

['[CLS]',
 'to',
 'compare',
 'the',
 'efficacy',
 'of',
 'metformin',
 'with',
 'insulin',
 'in',
 'diabetes',
 'mellitus',
 'in',
 'terms',
 'of',
 'fet',
 '##omat',
 '##ern',
 '##al',
 'outcome',
 '-',
 'discussion',
 'needed',
 'upon',
 'quality',
 'of',
 'the',
 'reporting',
 '.',
 '[SEP]']

In [32]:
len(example_text['ner_tags']), len(tokenized_input['input_ids'])

(25, 30)

#### testing tokenizer

In [33]:
q = tokenize_align_labels(abstract_hf_ds_train_test_valid['train'][0:1])
print(q)

{'input_ids': [[2, 1701, 4461, 1680, 3540, 1685, 12968, 1715, 3433, 1682, 3507, 7176, 1682, 4317, 1685, 4050, 7377, 20137, 1681, 3259, 16, 6372, 4291, 4188, 2993, 1685, 1680, 6888, 17, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 7, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [34]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]), q['labels'][0]):
    print(f'{token:_<40} {label}')


[CLS]___________________________________ -100
to______________________________________ 0
compare_________________________________ 0
the_____________________________________ 0
efficacy________________________________ 0
of______________________________________ 0
metformin_______________________________ 7
with____________________________________ 0
insulin_________________________________ 9
in______________________________________ 0
diabetes________________________________ 0
mellitus________________________________ 0
in______________________________________ 0
terms___________________________________ 0
of______________________________________ 0
fet_____________________________________ 0
##omat__________________________________ 0
##ern___________________________________ 0
##al____________________________________ 0
outcome_________________________________ 0
-_______________________________________ 0
discussion______________________________ 0
needed__________________________________ 0
upon____

#### tokenizing dataset

In [35]:
tokenized_dataset = abstract_hf_ds_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=abstract_hf_ds_train_test_valid['train'].column_names)

Map:   0%|          | 0/1239 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

#### Trainer

In [36]:
output_dir = r'/content/drive/MyDrive/colab_folder/models'
run_name = "Biolinkbert_Abstract"
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt

)

In [37]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
})

In [38]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023')

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.477900,0.251745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.234600,0.199328,0.000000,0.064286,0.000000,0.000000,0.000000,0.000000,0.532231,0.000000,0.440299,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.185800,0.196015,0.224852,0.500000,0.000000,0.576419,0.516213,0.301624,0.532784,0.254403,0.495946,0.168067,0.000000,0.000000,0.000000,0.000000
4,0.153700,0.179344,0.566845,0.585012,0.275510,0.608479,0.536364,0.298246,0.592129,0.303571,0.521181,0.273504,nan,nan,0.000000,0.000000
5,0.121700,0.176350,0.556364,0.589204,0.477124,0.621578,0.476954,0.336391,0.580801,0.344086,0.535121,0.253659,nan,nan,0.000000,0.000000
6,0.103600,0.203142,0.578947,0.621488,0.518950,0.633416,0.554930,0.342857,0.574692,0.357868,0.564722,0.265938,0.422535,0.085714,0.000000,0.000000
7,0.089800,0.197259,0.573290,0.604167,0.508380,0.645262,0.555882,0.360976,0.552469,0.348243,0.572443,0.258706,0.334448,0.277778,0.000000,0.000000
8,0.080500,0.204398,0.565289,0.602826,0.501425,0.633306,0.530864,0.365385,0.554054,0.362791,0.583739,0.244131,0.350168,0.142857,0.000000,0.000000


<ipython-input-17-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-17-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)


('/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/SWT_biolinkbert_tokenizer_15-11-2023/tokenizer.json')

#### Testing the model

In [39]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'] )

{'eval_loss': 0.21098501980304718,
 'eval_f1_B-cond': 0.6528925619834711,
 'eval_f1_I-cond': 0.5864939870490287,
 'eval_f1_B-des': 0.6025641025641025,
 'eval_f1_I-des': 0.6897163120567376,
 'eval_f1_B-subj': 0.4983388704318937,
 'eval_f1_I-subj': 0.4484536082474227,
 'eval_f1_B-group_A': 0.4487320837927233,
 'eval_f1_I-group_A': 0.3921085080147965,
 'eval_f1_B-group_B': 0.46680355920602323,
 'eval_f1_I-group_B': 0.3811129848229342,
 'eval_f1_B-group_C': 0.009216589861751152,
 'eval_f1_I-group_C': 0.08130081300813008,
 'eval_f1_B-group_D': 0.0,
 'eval_f1_I-group_D': 0.0,
 'eval_runtime': 2.0972,
 'eval_samples_per_second': 73.907,
 'eval_steps_per_second': 4.768,
 'epoch': 8.0}

In [40]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [41]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/checkpoint-390


In [42]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[{'entity_group': 'group_B', 'score': 0.42340177, 'word': 'semaglutide', 'start': 0, 'end': 11}, {'entity_group': 'group_B', 'score': 0.43190703, 'word': 'semaglutide', 'start': 159, 'end': 170}]


###  ABSTRACT AND METHODS

In [43]:
abstract_methods[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

##### count the annotations

In [44]:
for token, label in zip(abstract_methods[13]['tokens'], abstract_methods[13]['ner_tags']):
    print(f"{token:_<40} {label}")

alpha-dihydroergocryptine_______________ 7
in______________________________________ 0
the_____________________________________ 0
treatment_______________________________ 0
of______________________________________ 0
de______________________________________ 0
novo____________________________________ 0
parkinsonian____________________________ 0
patients________________________________ 5
:_______________________________________ 0
results_________________________________ 0
of______________________________________ 0
a_______________________________________ 0
multicentre_____________________________ 0
,_______________________________________ 0
randomized______________________________ 0
,_______________________________________ 0
double-blind____________________________ 0
,_______________________________________ 0
placebo-controlled______________________ 0
study___________________________________ 0
._______________________________________ 0
introduction____________________________ 0
:__________

In [45]:
tags = explore_annotations(abstract_methods)
import pandas as pd
result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 2158415 annotations in the dataset


,label_id,count
0,O,2102999
1,B-group_A,7941
2,B-subjects,7582
10,I-condition,7058
7,B-group_B,6515
5,I-design,6379
3,I-subjects,4317
6,I-group_A,3799
9,B-condition,3119
11,I-group_B,2982


##### Saving the data as a json

In [46]:
len(abstract_methods)

1549

#### Creating Huggingface Dataset

In [47]:
abstract_methods_hf_ds = create_datasets(abstract_methods, tru_label)

Casting the dataset:   0%|          | 0/1549 [00:00<?, ? examples/s]

In [48]:
abstract_methods_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [49]:
abstract_methods_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [50]:
_abstract_methods_part_train_test = train_test_split(abstract_methods_hf_ds, train_test_size=0.2, validation_size=0.5)


In [51]:
_abstract_methods_part_train_test

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
})

In [52]:
tags = _abstract_methods_part_train_test['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [53]:
tokenized_dataset = _abstract_methods_part_train_test.map(tokenize_align_labels, batched = True, remove_columns=_abstract_methods_part_train_test['train'].column_names)

Map:   0%|          | 0/1239 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

#### Trainer

In [54]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/biolinkbert/Abstract_Method'
run_name = "biolinkbert_Abstract_Method"
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt

)

In [55]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
})

In [56]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/Abstract_method_SWT_biolinkbert_model_18-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023')

Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.138800,0.122524,0.615730,0.673367,0.639511,0.781230,0.662791,0.590705,0.699408,0.576923,0.541538,0.527076,0.551471,0.395604,0.000000,0.000000
2,0.106800,0.126270,0.622074,0.677621,0.643326,0.778172,0.659772,0.576812,0.674169,0.554455,0.569925,0.602410,0.592834,0.409836,0.000000,0.000000
3,0.085600,0.141810,0.658363,0.664671,0.658174,0.758216,0.663443,0.577246,0.655089,0.572246,0.606916,0.574338,0.506122,0.602410,0.000000,0.000000
4,0.071500,0.147155,0.661792,0.640364,0.651584,0.773635,0.670873,0.591472,0.647382,0.545736,0.569620,0.540070,0.535519,0.500000,0.000000,0.000000
5,0.057800,0.154655,0.655113,0.612466,0.659091,0.771084,0.653465,0.619647,0.681070,0.514851,0.556380,0.553785,0.608108,0.440367,nan,0.000000
6,0.049300,0.157602,0.622296,0.676800,0.646681,0.759975,0.656995,0.598758,0.708764,0.548124,0.546495,0.500000,0.602007,0.217391,0.000000,0.000000
7,0.044000,0.167524,0.669052,0.648053,0.649237,0.766601,0.666667,0.605985,0.684426,0.566038,0.551775,0.523466,0.632258,0.375940,0.000000,nan
8,0.040000,0.170245,0.651489,0.627758,0.652361,0.759283,0.664596,0.605505,0.688406,0.547434,0.556943,0.526690,0.612378,0.358621,nan,nan


<ipython-input-17-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-17-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-17-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)


('/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/models/biolinkbert/tokenizer/Abstract_method_SWT_biolinkbert_tokenizer_18-11-2023/tokenizer.json')

#### Testing the model

In [57]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

{'eval_loss': 0.12873636186122894,
 'eval_f1_B-cond': 0.629324546952224,
 'eval_f1_I-cond': 0.5947955390334572,
 'eval_f1_B-des': 0.604206500956023,
 'eval_f1_I-des': 0.7452118398142773,
 'eval_f1_B-subj': 0.6137566137566138,
 'eval_f1_I-subj': 0.5992317541613316,
 'eval_f1_B-group_A': 0.7660434562910562,
 'eval_f1_I-group_A': 0.7768421052631579,
 'eval_f1_B-group_B': 0.6910074374577417,
 'eval_f1_I-group_B': 0.48826291079812195,
 'eval_f1_B-group_C': 0.7034883720930232,
 'eval_f1_I-group_C': 0.03636363636363636,
 'eval_f1_B-group_D': 0.0,
 'eval_f1_I-group_D': 0.0,
 'eval_runtime': 2.2989,
 'eval_samples_per_second': 67.425,
 'eval_steps_per_second': 4.35,
 'epoch': 8.0}

In [58]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [59]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/biolinkbert/Abstract_Method/checkpoint-78


In [60]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[{'entity_group': 'subj', 'score': 0.573934, 'word': 'patients', 'start': 127, 'end': 135}, {'entity_group': 'cond', 'score': 0.45220992, 'word': 'diabetes', 'start': 259, 'end': 267}]


###  ABSTRACT AND METHODS SPLITTED INTO CHUNKS OF 50

#### Creating a dataset for training

In [61]:
splitted_50_records_method_context[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['effects',
  'of',
  'rivastigmine',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'dementia',
  'associated',
  'with',
  'parkinson’s',
  'disease',
  '.',
  'we',
  'aimed',
  'to',
  'determine',
  'prospectively',
  'whether',
  'rivastigmine',
  ',',
  'an',
  'inhibitor',
  'of',
  'acetylcholinesterase',
  'and',
  'butyrylcholinesterase',
  ',',
  'provided',
  'benefits',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'a',
  'population',
  'with',
  'dementia',
  'associated',
  'with',
  "parkinson's",
  'disease',
  '(',
  'pdd',
  ')',
  '.',
  'this',
  

In [62]:
for token, label in zip(splitted_50_records_method_context[0]['tokens'], splitted_50_records_method_context[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

##### counting the annotation

In [63]:
tags = explore_annotations(splitted_50_records_method_context)

result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 2158415 annotations in the dataset


,label_id,count
0,O,2107343
1,B-group_A,7834
2,B-subjects,7421
5,B-group_B,6403
9,I-condition,5841
10,I-design,5321
3,I-subjects,3767
4,I-group_A,3543
8,B-condition,2909
11,I-group_B,2700


In [64]:
len(splitted_50_records_method_context)

35678

#### Creating Huggingface Dataset

In [65]:
splitted_50_records_method_context_hf_ds = create_datasets(splitted_50_records_method_context, tru_label)

Casting the dataset:   0%|          | 0/35678 [00:00<?, ? examples/s]

In [66]:
splitted_50_records_method_context_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [67]:
splitted_50_records_method_context_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['effects',
  'of',
  'rivastigmine',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'dementia',
  'associated',
  'with',
  'parkinson’s',
  'disease',
  '.',
  'we',
  'aimed',
  'to',
  'determine',
  'prospectively',
  'whether',
  'rivastigmine',
  ',',
  'an',
  'inhibitor',
  'of',
  'acetylcholinesterase',
  'and',
  'butyrylcholinesterase',
  ',',
  'provided',
  'benefits',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'a',
  'population',
  'with',
  'dementia',
  'associated',
  'with',
  "parkinson's",
  'disease',
  '(',
  'pdd',
  ')',
  '.',
  'this',
  

In [68]:
splitted_50_records_method_context_train_test_valid = train_test_split(splitted_50_records_method_context_hf_ds, train_test_size=0.2, validation_size=0.5)


In [69]:
splitted_50_records_method_context_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 28542
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3568
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3568
    })
})

In [70]:
tags = splitted_50_records_method_context_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [71]:
tokenized_dataset = splitted_50_records_method_context_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=splitted_50_records_method_context_train_test_valid['train'].column_names)

Map:   0%|          | 0/28542 [00:00<?, ? examples/s]

Map:   0%|          | 0/3568 [00:00<?, ? examples/s]

Map:   0%|          | 0/3568 [00:00<?, ? examples/s]

#### Trainer

In [72]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/biolinkbert'
run_name = 'biolinkbert_50_method_abstract'
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt)


In [73]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 28542
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3568
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3568
    })
})

In [74]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/Abstract_method_50_SWT_biolinkbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023')

Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.080900,0.070608,0.420881,0.455256,0.594378,0.603101,0.427292,0.361404,0.577172,0.421053,0.547685,0.303907,0.223642,0.408759,0.000000,0.000000
2,0.061800,0.071146,0.504775,0.399833,0.583732,0.658943,0.485258,0.464722,0.669612,0.489130,0.618361,0.402899,0.428866,0.281124,0.268908,0.035714
3,0.047600,0.067408,0.512535,0.499620,0.556561,0.614493,0.342541,0.434783,0.677355,0.523596,0.631144,0.414239,0.478821,0.464789,0.301587,0.161290
4,0.035500,0.073168,0.548975,0.497251,0.593361,0.667122,0.537430,0.495394,0.687538,0.558673,0.625059,0.441426,0.588889,0.471831,0.670391,0.380952
5,0.025600,0.080201,0.509554,0.507508,0.556962,0.704142,0.527512,0.497551,0.678024,0.522565,0.636325,0.450161,0.545455,0.466019,0.642487,0.368421
6,0.018400,0.086893,0.515759,0.516937,0.595041,0.726310,0.546213,0.462963,0.693442,0.548881,0.656934,0.449686,0.619658,0.493243,0.721311,0.357143
7,0.013100,0.094756,0.513907,0.503840,0.582031,0.743544,0.553646,0.454229,0.703692,0.535088,0.659189,0.425532,0.611714,0.485050,0.702439,0.337349
8,0.009800,0.101294,0.504749,0.508982,0.584314,0.739365,0.546717,0.461538,0.705421,0.545029,0.657941,0.424749,0.621739,0.501672,0.738462,0.329114


('/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_50_SWT_biolinkbert_tokenizer_15-11-2023/tokenizer.json')

In [2]:
import pandas as pd
history = pd.DataFrame(trainer.state.log_history)
history

NameError: ignored

#### Testing the model

In [75]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

{'eval_loss': 0.06713395565748215,
 'eval_f1_B-cond': 0.5363881401617251,
 'eval_f1_I-cond': 0.50040749796251,
 'eval_f1_B-des': 0.5219399538106235,
 'eval_f1_I-des': 0.6,
 'eval_f1_B-subj': 0.363481228668942,
 'eval_f1_I-subj': 0.4260027662517289,
 'eval_f1_B-group_A': 0.6877046740101221,
 'eval_f1_I-group_A': 0.5285359801488834,
 'eval_f1_B-group_B': 0.6506550218340612,
 'eval_f1_I-group_B': 0.5449275362318841,
 'eval_f1_B-group_C': 0.5654450261780105,
 'eval_f1_I-group_C': 0.4716981132075472,
 'eval_f1_B-group_D': 0.45652173913043476,
 'eval_f1_I-group_D': 0.6153846153846153,
 'eval_runtime': 9.1629,
 'eval_samples_per_second': 389.396,
 'eval_steps_per_second': 24.337,
 'epoch': 8.0}

In [76]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [77]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/biolinkbert/checkpoint-5352


In [78]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[]


###  ABSTRACT AND METHODS SPLITTED INTO CHUNKS OF 500

#### Let create a data set with just condition, subject, design

In [79]:
splitted_500_abstract_methods_context[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [80]:
for token, label in zip(splitted_500_abstract_methods_context[0]['tokens'], splitted_500_abstract_methods_context[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

In [1]:

tags = explore_annotations(splitted_500_abstract_methods_context)

result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

NameError: ignored

##### Saving the data as a json

In [82]:
len(splitted_500_abstract_methods_context)

4170

#### Creating Huggingface Dataset

In [83]:
splitted_500_abstract_methods_context_hf_ds = create_datasets(splitted_500_abstract_methods_context, tru_label)

Casting the dataset:   0%|          | 0/4170 [00:00<?, ? examples/s]

In [84]:
splitted_500_abstract_methods_context_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [85]:
splitted_500_abstract_methods_context_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [86]:
splitted_500_abstract_methods_context_train_test_valid = train_test_split(splitted_500_abstract_methods_context_hf_ds, train_test_size=0.2, validation_size=0.5)


In [87]:
splitted_500_abstract_methods_context_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3336
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 417
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 417
    })
})

In [88]:
tags = splitted_500_abstract_methods_context_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [89]:
tokenized_dataset = splitted_500_abstract_methods_context_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=splitted_500_abstract_methods_context_train_test_valid['train'].column_names)

Map:   0%|          | 0/3336 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

#### Trainer

In [ ]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/biolinkbert'
run_name = 'biolinkbert_500_method_abstract'
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt)


In [ ]:
tokenized_dataset

In [ ]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/biolinkbert/Abstract_method_500_SWT_biolinkbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/tokenizer/biolinkbert/Abstract_method_500_SWT_biolinkbert_tokenizer_15-11-2023')

#### Testing the model

In [ ]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

In [ ]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

In [ ]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

In [ ]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)